# Day 18
https://adventofcode.com/2017/day/18

In [1]:
import aocd
data = aocd.get_data(year=2017, day=18)

In [2]:
import regex as re
from collections import deque
from operator import __add__, __mul__, __mod__

In [3]:
OPERATORS = {
    'add': __add__,
    'mul': __mul__,
    'mod': __mod__,
}

In [4]:
def read_value(value):
    try:
        return int(value)
    except ValueError:
        return value

In [5]:
def read_commands(text):
    return tuple(tuple(read_value(v) for v in line.split(' ')) for line in text.split('\n'))

In [6]:
class Program():
    
    def __init__(self, commands, prog_no=None):
        self.commands = commands
        self.position = 0
        self.registers = dict()
        if prog_no is not None:
            self.registers['p'] = prog_no
        self.paused = False
        self.queue = deque()
        self.other_queue = None
        self.messages_sent = 0
    
    @property
    def deadlock(self):
        return (self.paused and len(self.queue) == 0) or self.position > len(self.commands)
    
    def attach(self, other):
        self.other_queue = other.queue
        other.other_queue = self.queue
    
    def get(self, key):
        if isinstance(key, int):
            return key
        return self.registers.get(key, 0)
    
    def step(self):
        command, *args = self.commands[self.position]
        if command == 'snd':
            if self.other_queue is None:
                self.queue.append(self.get(args[0]))
            else:
                self.other_queue.append(self.get(args[0]))
            self.messages_sent += 1
        elif command == 'set':
            self.registers[args[0]] = self.get(args[1])
        elif command == 'rcv':
            if self.other_queue is None:
                if self.get(args[0]) != 0:
                    self.paused = True
            else:
                if len(self.queue) == 0:
                    self.paused = True
                else:
                    self.registers[args[0]] = self.queue.popleft()
        elif command == 'jgz':
            if self.get(args[0]) > 0:
                self.position += self.get(args[1]) - 1
        elif command in ('add', 'mul', 'mod'):
            operator = OPERATORS.get(command)
            self.registers[args[0]] = operator(self.get(args[0]), self.get(args[1]))
        
        self.position += (0 if self.paused else 1)
    
    def run(self):
        self.paused = False
        while (not self.paused) and self.position < len(self.commands):
            self.step()

In [11]:
def run_duet(commands):
    prog0 = Program(commands, 0)
    prog1 = Program(commands, 1)
    prog0.attach(prog1)
    
    while not (prog0.deadlock and prog1.deadlock):
        prog0.run()
        prog1.run()
    
    return prog1.messages_sent

In [12]:
commands = read_commands(data)
prog = Program(commands)
prog.run()
p1 = prog.queue.pop()
print(f'Part 1: {p1}')
p2 = run_duet(commands)
print(f'Part 2: {p2}')

Part 1: 8600
Part 2: 7239
